# Week 4 - Models and Experimentation

## Step 1 Training a model

For the purposes of this demo, we will be using this [adapted demo](https://www.datacamp.com/tutorial/xgboost-in-python) and training an XGBoost model, and then doing some experimentation and hyperparameter tuning.


If running this notebook locally, use the following steps to create virtual environment:
- Don't use past python 3.10
- To create virtual environment use "venv"

`python -m venv NAME`

- Try to avoid anaconda, poetry or similar package management platforms
- To install a package use pip

`python -m pip install <package-name>`

- once you are done working with this virtual environment, deactivate it with `deactivate`

### Install packages

In [6]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00


In [7]:
import xgboost as xgb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


### Import data

We will be using Diamonds dataset imported from Seaborn. It is also available on [Kaggle](https://www.kaggle.com/datasets/shivam2503/diamonds).

Read about the features by following the link. We will be predicting the price of diamonds.

In [8]:
diamonds = sns.load_dataset('diamonds')
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [9]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float64 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int64   
 7   x        53940 non-null  float64 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB


In [10]:
diamonds.shape

(53940, 10)

In [11]:
X,y = diamonds.drop('price', axis=1), diamonds[['price']]

# For the cut, color and clarity use pandas category to enable XGBoost ability to deal with categorical data.

X['cut'] = X['cut'].astype('category')
X['color'] = X['color'].astype('category')
X['clarity'] = X['clarity'].astype('category')

### Split the data and train a model

In [12]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

In [13]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}

n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:39:19] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [14]:
# Define evaluation metrics - Root Mean Squared Error

predictions = model.predict(dtest)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")

RMSE: 532.8838153117543


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:39:20] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


### Incorporate validation

In [15]:
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 100

# Create the validation set
evals = [(dtrain, "train"), (dtest, "validation")]

In [16]:
evals = [(dtrain, "train"), (dtest, "validation")]

model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=10,
)

[0]	train-rmse:2859.49097	validation-rmse:2851.62630


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:39:20] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[10]	train-rmse:550.99470	validation-rmse:571.16640
[20]	train-rmse:491.51435	validation-rmse:544.08058
[30]	train-rmse:464.38845	validation-rmse:537.01895
[40]	train-rmse:445.99106	validation-rmse:533.85127
[50]	train-rmse:430.36010	validation-rmse:532.90320
[60]	train-rmse:418.87898	validation-rmse:533.04629
[70]	train-rmse:409.66247	validation-rmse:533.58046
[80]	train-rmse:397.34048	validation-rmse:534.31963
[90]	train-rmse:389.94294	validation-rmse:532.61946
[99]	train-rmse:377.70831	validation-rmse:532.88383


In [17]:
# Incorporate early stopping
n = 10000


model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=50,
   # Activate early stopping
   early_stopping_rounds=50
)

[0]	train-rmse:2859.49097	validation-rmse:2851.62630


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:39:21] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[50]	train-rmse:430.36010	validation-rmse:532.90320
[100]	train-rmse:377.56825	validation-rmse:532.79980
[102]	train-rmse:376.20429	validation-rmse:532.59813


In [18]:
# Cross-validation

params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 1000

results = xgb.cv(
   params, dtrain,
   num_boost_round=n,
   nfold=5,
   early_stopping_rounds=20
)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:39:22] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [19]:
results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,2861.153015,8.266765,2861.773555,36.937516
1,2081.378004,5.534608,2084.973481,32.064109
2,1545.361682,3.287745,1553.681211,31.059209
3,1182.364236,3.585787,1192.464771,26.157805
4,941.828819,2.971779,958.467497,23.613538


In [20]:
best_rmse = results['test-rmse-mean'].min()

best_rmse

549.1039652582465

## Start W&B


- Login into your W&B profile using the code below
- Alternatively you can set environment variables. There are several env variables which you can set to change the behavior of W&B logging. The most important are:
    - WANDB_API_KEY - find this in your "Settings" section under your profile
    - WANDB_BASE_URL - this is the url of the W&B server

- Find your API Token in "Profile" -> "Setttings" in the W&B App



In [21]:
# Log in to your W&B account
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [22]:
# TO DO
# Start experiment tracking with W&B
# Do at least 5 experiments with various hyperparameters
# Choose any method for hyperparameter tuning: grid search, random search, bayesian search
# Describe your findings and what you see

### Grid Search

In [23]:
for learning_rate in [0.01, 0.1, 0.2]:
    for max_depth in [6, 9, 12]:
        params = {
            "objective": "reg:squarederror",
            "tree_method": "gpu_hist",
            "learning_rate": learning_rate,
            "max_depth": max_depth
        }
        # Initialize a W&B run
        with wandb.init(project='diamonds_price_prediction', config=params):
            model = xgb.train(
                params=params,
                dtrain=dtrain,
                num_boost_round=100,
                evals=evals,
                early_stopping_rounds=50
            )
            predictions = model.predict(dtest)
            rmse = mean_squared_error(y_test, predictions, squared=False)
            wandb.log({"rmse": rmse})


wandb: Currently logged in as: saiganeshnellore (nsg). Use `wandb login --relogin` to force relogin


[0]	train-rmse:3951.92462	validation-rmse:3949.02748
[1]	train-rmse:3914.29025	validation-rmse:3911.21998
[2]	train-rmse:3877.10275	validation-rmse:3873.86672
[3]	train-rmse:3840.30732	validation-rmse:3836.90210
[4]	train-rmse:3803.89438	validation-rmse:3800.32256
[5]	train-rmse:3767.86467	validation-rmse:3764.12914
[6]	train-rmse:3732.20828	validation-rmse:3728.30718
[7]	train-rmse:3696.92823	validation-rmse:3692.85775
[8]	train-rmse:3662.02397	validation-rmse:3657.78952
[9]	train-rmse:3627.46912	validation-rmse:3623.08587
[10]	train-rmse:3593.28635	validation-rmse:3588.73717
[11]	train-rmse:3559.44161	validation-rmse:3554.71475
[12]	train-rmse:3525.97874	validation-rmse:3521.06781
[13]	train-rmse:3492.86570	validation-rmse:3487.77529
[14]	train-rmse:3460.11145	validation-rmse:3454.83778
[15]	train-rmse:3427.65383	validation-rmse:3422.21992
[16]	train-rmse:3395.56677	validation-rmse:3389.96690


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:40:15] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[17]	train-rmse:3363.75667	validation-rmse:3358.08757
[18]	train-rmse:3332.31965	validation-rmse:3326.47775
[19]	train-rmse:3301.17180	validation-rmse:3295.26749
[20]	train-rmse:3270.41375	validation-rmse:3264.35438
[21]	train-rmse:3239.92519	validation-rmse:3233.88165
[22]	train-rmse:3209.73054	validation-rmse:3203.59201
[23]	train-rmse:3179.80215	validation-rmse:3173.60675
[24]	train-rmse:3150.18892	validation-rmse:3144.01912
[25]	train-rmse:3120.92669	validation-rmse:3114.64985
[26]	train-rmse:3091.93693	validation-rmse:3085.54884
[27]	train-rmse:3063.14791	validation-rmse:3056.74740
[28]	train-rmse:3034.75572	validation-rmse:3028.31985
[29]	train-rmse:3006.57676	validation-rmse:3000.08035
[30]	train-rmse:2978.78205	validation-rmse:2972.18011
[31]	train-rmse:2951.11499	validation-rmse:2944.49426
[32]	train-rmse:2923.86171	validation-rmse:2917.20940
[33]	train-rmse:2896.92357	validation-rmse:2890.16929
[34]	train-rmse:2870.06897	validation-rmse:2863.29618
[35]	train-rmse:2843.69881	v

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:40:16] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


...

[0]	train-rmse:3217.83343	validation-rmse:3214.66438
[1]	train-rmse:2602.13630	validation-rmse:2600.82904
[2]	train-rmse:2111.72985	validation-rmse:2114.16783


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:41:36] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[3]	train-rmse:1721.45750	validation-rmse:1730.05651
[4]	train-rmse:1411.27899	validation-rmse:1428.59849
[5]	train-rmse:1165.15099	validation-rmse:1193.70312
[6]	train-rmse:970.54790	validation-rmse:1012.80740
[7]	train-rmse:817.76823	validation-rmse:877.60149
[8]	train-rmse:697.63489	validation-rmse:777.17345
[9]	train-rmse:603.72056	validation-rmse:706.05451
[10]	train-rmse:531.87223	validation-rmse:653.97026
[11]	train-rmse:474.52661	validation-rmse:619.87326
[12]	train-rmse:430.74522	validation-rmse:593.90690
[13]	train-rmse:394.22816	validation-rmse:576.30086
[14]	train-rmse:366.76698	validation-rmse:566.23577
[15]	train-rmse:345.24781	validation-rmse:558.59428
[16]	train-rmse:324.30737	validation-rmse:553.38875
[17]	train-rmse:306.56395	validation-rmse:550.49999
[18]	train-rmse:296.13579	validation-rmse:548.25369
[19]	train-rmse:284.62208	validation-rmse:547.41576
[20]	train-rmse:273.14734	validation-rmse:546.54901
[21]	train-rmse:264.49993	validation-rmse:545.21005
[22]	train-r

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:41:39] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


rmse,▁
rmse,552.90571


Observations on Model Performance:
RMSE Trends: Throughout the experiments, the Root Mean Square Error (RMSE) generally decreased as the models continued training, which indicates that the models were learning effectively from the training data.

Validation Set Performance: The RMSE on the validation set showed slight fluctuations, suggesting a need to monitor for overfitting or to adjust model complexity.

**Impact of Hyperparameters:**

Changes in learning rate, max depth, and other parameters significantly influenced model performance. For instance, lower learning rates might have caused slower convergence, whereas higher rates might lead to overshooting the minimum.

Important parameters which had more influence is learning_rate


The number of boosting rounds also played a crucial role. In some runs, increasing the rounds continued to improve the RMSE, suggesting that further training was beneficial up to a point.



rmse:524.1966314359638
*   learning_rate:0.1
*   max_depth:6




### Random Search

In [24]:
import random

for _ in range(5):
    params = {
        "objective": "reg:squarederror",
        "tree_method": "gpu_hist",
        "learning_rate": random.uniform(0.01, 0.2),
        "max_depth": random.randint(6, 12)
    }
    with wandb.init(project='diamonds_price_prediction', config=params):
        model = xgb.train(
            params=params,
            dtrain=dtrain,
            num_boost_round=100,
            evals=evals,
            early_stopping_rounds=50
        )
        predictions = model.predict(dtest)
        rmse = mean_squared_error(y_test, predictions, squared=False)
        wandb.log({"rmse": rmse})


[0]	train-rmse:3630.21272	validation-rmse:3626.49109
[1]	train-rmse:3304.93808	validation-rmse:3300.23857
[2]	train-rmse:3010.81222	validation-rmse:3005.32242
[3]	train-rmse:2744.70338	validation-rmse:2738.95184
[4]	train-rmse:2504.05344	validation-rmse:2498.85603
[5]	train-rmse:2286.42476	validation-rmse:2282.12875
[6]	train-rmse:2089.18432	validation-rmse:2085.81751
[7]	train-rmse:1910.67143	validation-rmse:1907.96545


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:41:45] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[8]	train-rmse:1750.08958	validation-rmse:1748.52848
[9]	train-rmse:1605.48924	validation-rmse:1605.24593
[10]	train-rmse:1475.17870	validation-rmse:1475.51489
[11]	train-rmse:1357.71560	validation-rmse:1359.08053
[12]	train-rmse:1252.60160	validation-rmse:1256.20355
[13]	train-rmse:1158.67783	validation-rmse:1164.19042
[14]	train-rmse:1074.43494	validation-rmse:1082.44751
[15]	train-rmse:999.43192	validation-rmse:1010.14866
[16]	train-rmse:932.55408	validation-rmse:945.63031
[17]	train-rmse:872.81700	validation-rmse:888.78185
[18]	train-rmse:819.99626	validation-rmse:838.31408
[19]	train-rmse:772.96456	validation-rmse:794.27974
[20]	train-rmse:731.20176	validation-rmse:755.09995
[21]	train-rmse:694.75281	validation-rmse:722.18920
[22]	train-rmse:663.05025	validation-rmse:692.88491
[23]	train-rmse:634.65037	validation-rmse:667.92279
[24]	train-rmse:609.87346	validation-rmse:645.90324
[25]	train-rmse:588.06363	validation-rmse:627.48136
[26]	train-rmse:569.56937	validation-rmse:612.01187

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:41:47] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


rmse,▁
rmse,523.11639


...

[0]	train-rmse:3784.38376	validation-rmse:3780.99540
[1]	train-rmse:3590.34810	validation-rmse:3586.00062
[2]	train-rmse:3407.05510	validation-rmse:3402.04952
[3]	train-rmse:3233.97807	validation-rmse:3228.18009
[4]	train-rmse:3070.14862	validation-rmse:3064.69086
[5]	train-rmse:2915.61763	validation-rmse:2909.89417
[6]	train-rmse:2769.34484	validation-rmse:2763.86727
[7]	train-rmse:2631.29351	validation-rmse:2625.69169
[8]	train-rmse:2499.95997	validation-rmse:2494.82788
[9]	train-rmse:2376.55585	validation-rmse:2371.59413
[10]	train-rmse:2259.61728	validation-rmse:2254.96517
[11]	train-rmse:2149.06048	validation-rmse:2144.16968
[12]	train-rmse:2044.97187	validation-rmse:2040.51982


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:42:20] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[13]	train-rmse:1946.32173	validation-rmse:1941.90833
[14]	train-rmse:1853.72247	validation-rmse:1850.01058
[15]	train-rmse:1766.19805	validation-rmse:1762.85120
[16]	train-rmse:1683.61329	validation-rmse:1680.72754
[17]	train-rmse:1605.84618	validation-rmse:1602.78250
[18]	train-rmse:1532.62072	validation-rmse:1529.74929
[19]	train-rmse:1463.43252	validation-rmse:1461.00849
[20]	train-rmse:1398.25382	validation-rmse:1395.91428
[21]	train-rmse:1337.27054	validation-rmse:1335.29151
[22]	train-rmse:1279.64429	validation-rmse:1278.52445
[23]	train-rmse:1225.41440	validation-rmse:1224.99272
[24]	train-rmse:1174.62224	validation-rmse:1174.83223
[25]	train-rmse:1126.49389	validation-rmse:1127.50385
[26]	train-rmse:1081.72254	validation-rmse:1083.47990
[27]	train-rmse:1039.55611	validation-rmse:1042.31902
[28]	train-rmse:1000.27805	validation-rmse:1003.67902
[29]	train-rmse:963.27748	validation-rmse:967.52703
[30]	train-rmse:928.75300	validation-rmse:934.06362
[31]	train-rmse:896.49488	valida

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [22:42:21] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


rmse,▁
rmse,527.32834


#### Hyperparameter W&B Sweeps with Bayes Method

In [29]:
import wandb
wandb.login()

sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'rmse',
        'goal': 'minimize'
    },
    'parameters': {
        'max_depth': {
            'values': [3, 6, 9, 12]
        },
        'learning_rate': {
            'values': [0.01, 0.1, 0.2]
        },
        'n_estimators': {
            'values': [100, 200, 300]
        },
        'subsample': {
            'values': [0.8, 0.9, 1.0]
        },
        'colsample_bytree': {
            'values': [0.5, 0.75, 1.0]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="diamonds_price_prediction")


Create sweep with ID: yhuzk2ul
Sweep URL: https://wandb.ai/nsg/diamonds_price_prediction/sweeps/yhuzk2ul


In [30]:
def train():
    wandb.init()

    # Load dataset and preprocess
    diamonds = sns.load_dataset('diamonds')
    X = diamonds.drop('price', axis=1)
    y = diamonds['price']
    X['cut'] = X['cut'].astype('category')
    X['color'] = X['color'].astype('category')
    X['clarity'] = X['clarity'].astype('category')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
    dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

    # Get hyperparameters from W&B
    config = wandb.config

    params = {
        "objective": "reg:squarederror",
        "tree_method": "gpu_hist",
        "max_depth": config.max_depth,
        "learning_rate": config.learning_rate,
        "n_estimators": config.n_estimators,
        "subsample": config.subsample,
        "colsample_bytree": config.colsample_bytree
    }

    # Train model
    model = xgb.train(params, dtrain, num_boost_round=100)

    # Predict and calculate RMSE
    predictions = model.predict(dtest)
    rmse = mean_squared_error(y_test, predictions, squared=False)

    # Log the RMSE to W&B
    wandb.log({"rmse": rmse})

    wandb.finish()


In [32]:
wandb.agent(sweep_id, train, count=25)

wandb: Agent Starting Run: qd8lopzs with config:
wandb: 	colsample_bytree: 0.75
wandb: 	learning_rate: 0.01
wandb: 	max_depth: 12
wandb: 	n_estimators: 200
wandb: 	subsample: 0.8


rmse,▁
rmse,1623.08621


wandb: Agent Starting Run: rvazewkj with config:
wandb: 	colsample_bytree: 1
wandb: 	learning_rate: 0.1
wandb: 	max_depth: 3
wandb: 	n_estimators: 200
wandb: 	subsample: 0.9


rmse,▁
rmse,612.80815


wandb: Agent Starting Run: 6qkxk4l5 with config:
wandb: 	colsample_bytree: 0.5
wandb: 	learning_rate: 0.2
wandb: 	max_depth: 9
wandb: 	n_estimators: 300
wandb: 	subsample: 0.9


...

rmse,▁
rmse,580.22251


wandb: Agent Starting Run: qak5bqbv with config:
wandb: 	colsample_bytree: 1
wandb: 	learning_rate: 0.2
wandb: 	max_depth: 3
wandb: 	n_estimators: 200
wandb: 	subsample: 0.9


rmse,▁
rmse,587.47154


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 7wu3jp6g with config:
wandb: 	colsample_bytree: 0.75
wandb: 	learning_rate: 0.2
wandb: 	max_depth: 3
wandb: 	n_estimators: 200
wandb: 	subsample: 0.8


rmse,▁
rmse,580.22251


## Experiment Observation and Analysis

### Objective
The goal of this series of experiments was to optimize the hyperparameters of an XGBoost model trained on the Diamonds dataset. We aimed to minimize the root mean squared error (RMSE) of the model's predictions on the test dataset. We used Weights & Biases (W&B) for experiment tracking and hyperparameter sweeps.

### Methodology
We employed W&B's Bayesian optimization for the hyperparameter tuning process, focusing on several key parameters:
- `max_depth`: The maximum depth of the trees.
- `learning_rate`: The step size shrinkage used to prevent overfitting.
- `n_estimators`: The number of trees in the forest.
- `subsample`: The fraction of samples to be used for fitting the individual base learners.
- `colsample_bytree`: The fraction of features to be used for each tree.

These parameters were varied over defined ranges to observe their effect on the RMSE.

### Results and Insights
The experiments were conducted over multiple runs, each with a different set of hyperparameters. Here are some of the noteworthy observations:

- The lowest RMSE achieved was 529.78, with the hyperparameters set to `colsample_bytree: 0.75`, `learning_rate: 0.2`, `max_depth: 6`, `n_estimators: 300`, and `subsample: 1.0`.
- Increasing the `max_depth` and `n_estimators` generally led to better performance, indicating that more complex models were able to capture the underlying patterns in the data more effectively.
- The `learning_rate` at 0.2 combined with higher values of `max_depth` and `n_estimators` consistently yielded better results, suggesting that a higher learning rate works well with a more complex model in this dataset.
- The optimal `subsample` and `colsample_bytree` values hovered around 0.9 and 0.75, respectively, balancing the model's ability to train on diverse data samples while avoiding overfitting.